# スケジュール算出プログラム

## 1.ライブラリのインポート
pandasは表の処理

In [68]:
import random
import datetime
import numpy as np
import pandas as pd
from array import array


## 2.時間のローカライズ

In [69]:
def read_excels():

    excel_path = "f_iden_3.xlsm" 
    


    off_day_df  = pd.read_excel(excel_path, sheet_name="yukyu")                         # 休みの希望表
    skill_df    = pd.read_excel(excel_path, sheet_name="skill", header=[0, 1])          # スキル表
    due_date_df = pd.read_excel(excel_path, sheet_name="calendar")                      # 納期表
    start_day:pd.Series = pd.read_excel(excel_path, sheet_name="start")    

    # 空データ(NaN)の置換(0)
    off_day_df.fillna(0, inplace=True)

    due_date_df.fillna(0, inplace=True)

    # データの置換
    off_day_df.replace("◎", 2, inplace=True)                                            # 希望休（◎）を 2 に置換
    due_date_df.replace("☆", 3, inplace=True)                                          # 納期（☆）を 3 に置換

    # 列ごとのデータ型(dtype)を、intに変更しておく
    off_day_df[off_day_df.columns[2:]] = off_day_df[off_day_df.columns[2:]].astype("int64")
    due_date_df[due_date_df.columns[5:]] = due_date_df[due_date_df.columns[5:]].astype("int64")   #スキルグループ→工程による列追加

    # 品番の抽出
    product_num = due_date_df.loc[:, "品番"]

    # 土日の削除
    # データの中から土日の日付を取り出す
    off_days = [date for date in due_date_df.columns[5:] if date.strftime("%a") in ["Sat","Sun"]]  

    # 土日の日付を削除する
    off_day_df.drop(columns=off_days, inplace=True)
    due_date_df.drop(columns=off_days, inplace=True)

    return off_day_df, skill_df, due_date_df, product_num , start_day

In [70]:
off_day_df, skill_df, due_date_df, product_num , start_day = read_excels()

In [82]:

#追加
p_start=pd.to_datetime(start_day.iloc[:,1])
type(p_start)
p_start


Series([], Name: 2023-06-02 00:00:00, dtype: datetime64[ns])

## 3.第1世代関数

In [72]:
def first_gen(off_day_df: pd.DataFrame, skill_df: pd.DataFrame, due_date_df: pd.DataFrame, product_num: pd.Series):
    
    # 調整後の納期表（納期表のコピー）　
    adjusted_due_date_df = due_date_df.copy()


    # 作業者の割り振り　と　作業者の希望休を調整納期表にコピー
    worker_list = []
    # indexごとにrowを取り出して、作業者をランダムに割り振る
    for index, row in adjusted_due_date_df.iterrows():



        workers: pd.DataFrame = skill_df.loc[:, (["スキルグループ"], [row["スキルグループ"]])] 
        worker = workers.dropna().sample().values[0, 0]

        worker_list.append(worker)
        worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]), off_day_df.columns[2:]]                   # 作業者の希望休を抽出


        adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[5:]] = worker_off_days.values   
 
    # 割り当てたワーカーを adjusted_due_date_df へ「作業者」列として追加（※ここから adjusted_due_date_df が一列増えるので注意）
    adjusted_due_date_df.insert(3, "作業者", worker_list)

   
    # 作業者の希望休に合わせて、納期を調整
    after_index = None                                                                                                              # 後工程の index を保持する
    
    for index, row in adjusted_due_date_df[::-1].iterrows():                                                                        # due_date_df を逆順で行ごとに処理

        # 現工程の納期を抽出
        process: pd.Series = due_date_df.loc[index, due_date_df.columns[5:]]                                                        

        process_due_date = process[process.isin([3])]                                                                               # 現工程の納期
        #  print("入力納期:", process_due_date.index[0])
        #  print("入力納期process_due_date:", process_due_date)

        if after_index != None:

            after_process_df: pd.DataFrame = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == row["製造番号"]]                               # 同じ製造番号のタスクを取り出す。


            #print("[INFO]: 工程:", row["工程"]) 


            if row["工程"] + 1 in after_process_df["工程"].values:

                #print("後工程あり")


                after_index = after_process_df[after_process_df["工程"].isin([row["工程"] + 1])].index[0]
                #print("after_index :", after_index)

                # 後工程の納期を抽出
                after_process: pd.Series = adjusted_due_date_df.loc[after_index, adjusted_due_date_df.columns[6:]]                 #スキルグループ→工程による列追加
                after_process_due_date = after_process[after_process.isin([3])]                                                     # 後工程の納期
                #print("後工程の納期:", after_process_due_date.index[0])


                if not process_due_date.index[0] < after_process_due_date.index[0]:

                    # 納期を１営業日早めて調整する（この時点では、有休をチェックしてないので決定しない）
                    before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[5]:after_process_due_date.index[0]]     # 現在の納期より前のデータを抽出

                    new_due_date = before_days.columns[-2]                                                                          # 納期を１営業日早める
                    #print("後工程を考慮した納期調整:", new_due_date)
                    
                    due_date_df.loc[index, process_due_date.index[0]] = 0                                                           # 現在の納期を納期表から削除
                    due_date_df.loc[index, new_due_date] = process_due_date.values[0]                                               # 新しい納期を納期表に追加

                    process: pd.Seies = due_date_df.loc[index, due_date_df.columns[5:]]                                            

                    process_due_date = process[process.isin([3])]                                                                   # 現工程の納期
                    #print("現工程の納期:", process_due_date.index[0])

            else:
                pass
                #print("後工程なし:")
        
        else:
            pass
            #print("後工程なし（1つ目のデータ）:")
        
        
        # 希望休に合わせた調整
        # 現在の納期より前のデータを抽出
        before_days: pd.DataFrame = adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[6]:process_due_date.index[0]] #スキルグループ→工程による列追加


        for new_due_date in before_days.columns[::-1]:
                        
            # 納期を１日早める(-1日)
            #print("納期候補:", new_due_date)

            # その日が希望休でない場合(0 のとき)　→　納期決定　→　ループ終了
            if adjusted_due_date_df.loc[index, new_due_date] == 0:

                # 納期が確定するのはココだけ。valuesで[3]が入るindexと配列指定。values[0]で3指定。
                #print("納期決定:", new_due_date)
                adjusted_due_date_df.loc[index, new_due_date] = process_due_date.values[0]
                break
        
        after_index = index
        #print("-" * 100)              # デバッグ用（ループごとに横線を表示）

    return adjusted_due_date_df


In [73]:
#受取
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

In [74]:
#第1世代

## ※　データのルール
製造番号は必ず連番

In [75]:
#adjusted_due_date_df[adjusted_due_date_df["製造番号"] == 1].iloc[:, :15]

## 4.納期をランダムで振り直し（再帰処理）
※　この処理中は希望休を考慮しない


### 開始日

In [76]:
#start_date = datetime.datetime(2023, 6, 2)
start_date= p_start

### 納期のランダム振り分け関数

In [77]:
def generate_random_dates(
        start_date: datetime.datetime, 
        end_date: datetime.datetime):
    
    start_date = due_date_df.loc[:, start_date:].columns[1]

    working_days = due_date_df.loc[:, start_date:end_date].columns
    #print(working_days)

    random_num = random.randint(0, working_days.shape[0]-1)
    random_date = working_days[random_num]
    #print(random_date)
    #print("-" * 50, "納期ランダム割り振り完了","-" * 50)

    return random_date

### 同じ製造番号のタスクをランダムにリスケする関数（これが再帰関数：関数内の処理に自分自身の呼び出しがある）

In [78]:
def random_due_date(
        start_date: datetime.datetime,
        process_df: pd.DataFrame,
        process_num: int):
    
    #print("process_num :", process_num, "-" * 100)
    #print('process_df',process_df.loc[[process_df.index[process_num]], process_df.columns[:5]])

    process: pd.Series = process_df.loc[process_df.index[process_num], process_df.columns[5:]]              # 現工程の納期表データ
    deadline = process[process.isin([3])]                                                                   # 現工程の納期
    deadline_date: datetime.datetime = deadline.index[0]                                                    # その工程のデッドライン
    #print("process_deadline :", deadline_date)


    if process_num != 0:

        start_date = random_due_date(start_date, process_df.loc[process_df.index[:-1], :], process_num-1)   #（再帰処理）

    new_due_date = generate_random_dates(start_date, deadline_date)                                         # 開始日～デッドライン　の期間からランダムに日付を選択

    due_date_df.loc[process_df.index[process_num], deadline.index[0]] = 0                                   # 現在の納期を納期表から削除
    due_date_df.loc[process_df.index[process_num], new_due_date] = deadline.values[0]                       

    return new_due_date
                     


### タスクを製造番号ごとに処理する関数

In [79]:
def change_due_date():
    
    serial_number = due_date_df.loc[:, "製造番号"].max()
    #print('serial_number',serial_number)

    # 製造番号ごとに繰り返す
    for i in range(1, serial_number + 1):

        # 同じ製造番号のタスクを取り出す。
        process_df = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i]

        #print('process_df.shape',process_df.shape)
        #print('process_df.loc[:, process_df.columns].isin([3])')



        random_due_date(start_date, process_df, process_df.shape[0]-1)                                          # 製造番号ごとに納期のランダム割り当て（再帰関数）

        #print('タスク',process_df)
        #print('process_df.shape[0]',process_df.shape[0])
        #print("*" * 200)

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。



In [80]:
# 呼び出し
change_due_date()

InvalidIndexError: Series([], Name: 2023-06-02 00:00:00, dtype: datetime64[ns])

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。

In [ ]:
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

#### これでランダムな納期かつ希望休の調整ができる

##### 希望休表

##### 希望休調整前(ランダム)

##### 希望休調整後

In [ ]:
adjusted_due_date_df_copy= adjusted_due_date_df.copy()
adjusted_due_date_df_copy=adjusted_due_date_df_copy.iloc[:,5:]
adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
#行と列の数を取り出す
sh3=adjusted_due_date_df_copy.shape
adjusted_due_date_df_copy.columns=range(sh3[1])


In [ ]:
#45変換
def pre_evalution(adjusted_due_date_df):
    adjusted_due_date_df2=adjusted_due_date_df.iloc[:,6:] #スキルグループ→工程による列追加

    return adjusted_due_date_df2

In [ ]:
#受取
adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)

In [ ]:
x=adjusted_due_date_df2.replace(3,'☆').replace(2,'◎').replace(0,'')
#x.to_excel('AS_tochu.xlsx')
x

,2023-06-01,2023-06-02,2023-06-05,2023-06-06,2023-06-07,2023-06-08,2023-06-09,2023-06-12,2023-06-13,2023-06-14,...,2023-06-19,2023-06-20,2023-06-21,2023-06-22,2023-06-23,2023-06-26,2023-06-27,2023-06-28,2023-06-29,2023-06-30
0,,,☆,◎,,,,,,,...,,,,,,,,,,
1,,,,☆,,,,,,,...,,,,,,,,,,
2,,,,,,☆,,,,,...,◎,,,,,◎,,,,
3,,,,,,,☆,◎,◎,,...,,,,,,,,,,
4,,,,,,,,,,,...,☆,,,,,,,,,
5,,,,,,,◎,,,,...,,☆,,,,,,,,
6,,,,,,,,,,,...,,,☆,◎,,,,,,
7,,,☆,,,,,,,,...,,,,,,,,,,
8,,,,,,,,◎,◎,,...,☆,,,,,,,,,
9,,,,,,,,,,,...,,☆,,,,,,,,


In [ ]:
#追加
due_date_df_copy= due_date_df.copy()
due_date_df_copy.loc[due_date_df.index, due_date_df.columns[5:]] = x.values
due_date_df_copy= due_date_df_copy.set_index('製造番号')
date_columns = pd.to_datetime(due_date_df_copy.columns[4:])
formatted_dates = date_columns.strftime('%m/%d')
#print(formatted_dates)
due_date_df_copy.columns = list(due_date_df_copy.columns[:4]) + list(formatted_dates)
date_column = pd.to_datetime(due_date_df_copy['納期'])
formatted_dates2 = date_column.dt.strftime('%m/%d')
# date_index = pd.to_datetime(due_date_df_copy.loc['納期'])
# formatted_dates2 = date_index.strftime('%m/%d')
#print(formatted_dates2)
due_date_df_copy['納期']= formatted_dates2
due_date_df_copy

C:\Users\r27150042\AppData\Local\Temp\ipykernel_12728\3903163671.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  due_date_df_copy.loc[due_date_df.index, due_date_df.columns[5:]] = x.values
C:\Users\r27150042\AppData\Local\Temp\ipykernel_12728\3903163671.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['☆', '', '', '', '', '', '', '☆', '', '', '', '', '', '', '☆', '', '☆', '', '', '', '', '', '', '', '', '', '', '☆', '', '', '', '', '', '', '', '', '', '☆', '', '', '☆', '', '☆', '', '', '', '', '', '☆', '', '', '']' has dtype incompatible with 

,品番,工程,スキルグループ,納期,06/01,06/02,06/05,06/06,06/07,06/08,...,06/19,06/20,06/21,06/22,06/23,06/26,06/27,06/28,06/29,06/30
製造番号,,,,,,,,,,,,,,,,,,,,,
1,16204C49450,1,1,06/12,,,☆,◎,,,...,,,,,,,,,,
1,16204C49450,2,2,06/12,,,,☆,,,...,,,,,,,,,,
1,16204C49450,3,3,06/12,,,,,,☆,...,◎,,,,,◎,,,,
2,16205G59900-1,1,1,06/22,,,,,,,...,,,,,,,,,,
2,16205G59900-1,2,2,06/22,,,,,,,...,☆,,,,,,,,,
2,16205G59900-1,3,3,06/22,,,,,,,...,,☆,,,,,,,,
2,16205G59900-1,4,4,06/22,,,,,,,...,,,☆,◎,,,,,,
3,1J07071011,1,1,06/30,,,☆,,,,...,,,,,,,,,,
4,1J43171472B,1,1,06/26,,,,,,,...,☆,,,,,,,,,


In [ ]:
due_date_df_copy.to_excel('AS_tochu2.xlsx')